<a href="https://colab.research.google.com/github/eluisluzquadros/covid_impact_se/blob/main/step01_2_collection_white_ScraperGoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [1]:

#!apt-get update # to update ubuntu to correctly run apt install

# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3




Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
cp: missing destination file operand after '/usr/lib/chromium-browser/chromedriver/usr/bin'
Try 'cp --help' for more information.
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.te

### Import das dependencias

In [100]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [101]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/02_white/2021/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

### Import chaves de api

In [ ]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impactSE/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [10]:
import pandas as pd
GSE_RESULTS_CLASS = 'gs_r gs_or gs_scl' #'ZINbbc xpd O9g5cc uUPGi'
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  d_before = (date + DateOffset(days = 2)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://scholar.google.com/scholar?hl=fr&as_sdt=0%2C5&q={}&sort=date:r:{}:{}&num={}".format(q, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = " + "after:" + date.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', '').split('&sa')[0],
            
    }
    
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [11]:
# Scraper 1 (2020)
keyword = '"covid impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"' 


# Scraper 3 (2021 v2)
# keyword = '"covid-19" OR "covid" OR "coronavirus" AND “impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "agile" OR "information industry" OR "IT" OR "disaster informatics" OR "crisis informatics" OR "productivity" OR "remote" OR "work" OR "future of work"'

### Merge csv's or SCRAPER new search

In [12]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])
else:
  #get first date
  first_date = date(2021, 1, 1)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  df = fetch_ipaddress(df)
  #get geocoded ipaddress
  df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
29,29,2021-01-02,21 tech leaders predict what 2021 could be lik...,"Jan 2, 2021 · But in that gloom, one industry ...",https://timesofindia.indiatimes.com/business/i...,https,timesofindia.indiatimes.com,/business/india-business/21-tech-leaders-predi...,NaN,NaN,indiatimes.com,23.44.109.229,US,United States,Georgia,Atlanta,-84.388283,33.798458,16625.0,NaN,geolocation,{},23.44.109.229,21 tech leaders predict what 2021 could be lik...,"Jan 2, 2021 · But in that gloom, one industry ..."
30,30,2021-01-02,Variable Indicator - an overview | ScienceDire...,Kitchenham and Mendes [37] used stepwise regre...,https://www.sciencedirect.com/topics/computer-...,https,www.sciencedirect.com,/topics/computer-science/variable-indicator,NaN,NaN,sciencedirect.com,162.159.130.81,US,United States,Illinois,Chicago,-87.671249,41.848850,13335.0,NaN,geolocation,{},162.159.130.81,Variable Indicator - an overview | ScienceDire...,Kitchenham and Mendes [37] used stepwise regre...
31,31,2021-01-02,Core providers can help accelerate Robotics Pr...,"Jan 4, 2021 · RPA technology consists of softw...",https://www.fisglobal.com/en/insights/what-we-...,https,www.fisglobal.com,/en/insights/what-we-know/2021/january/core-pr...,NaN,NaN,fisglobal.com,104.90.46.157,US,United States,Georgia,Atlanta,-84.388283,33.798458,16625.0,NaN,geolocation,{},104.90.46.157,Core providers can help accelerate Robotics Pr...,"Jan 4, 2021 · RPA technology consists of softw..."
32,32,2021-01-02,Growth sensitivity to climate varies with soil...,"Jan 3, 2021 · The effects of climate change on...",https://link.springer.com/article/10.1007/s004...,https,link.springer.com,/article/10.1007/s00468-020-02066-8,NaN,NaN,springer.com,151.101.0.95,US,United States,California,San Francisco,-122.392860,37.767849,54113.0,NaN,geolocation,{},151.101.0.95,Growth sensitivity to climate varies with soil...,"Jan 3, 2021 · The effects of climate change on..."
33,33,2021-01-02,Building a resilient future for the BPO indust...,"Jan 4, 2021 · Many of the call centers were fo...",https://pldtenterprise.com/news-and-events/bpo...,https,pldtenterprise.com,/news-and-events/bpo/building-resilient-future...,NaN,NaN,pldtenterprise.com,45.60.158.66,US,United States,California,Los Angeles,-118.241638,34.045639,19551.0,NaN,geolocation,{},45.60.158.66,Building a resilient future for the BPO indust...,"Jan 4, 2021 · Many of the call centers were fo..."


### SCRAPER new search

In [89]:
#get next data
next_date = next_data(df)

#get seach
df_new = search(keyword, next_date)

#get domain
df_new = fetch_domain(df_new)

#get hostname
df_new = fetch_hostname(df_new)

#get ipaddress
df_new = fetch_ipaddress(df_new)

#get geocoded ipaddress
df_new = fetch_location(df_new)

#get nlp contextualSpellCheck 
df_new = fetch_nlp(df_new)

#save df new
save(df_new)

#print new df
df_new.head(2)



https://www.google.com/search?q="covid-19" OR "covid" OR "coronavirus" AND “impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "agile" OR "information industry" OR "IT" OR "disaster informatics" OR "crisis informatics" OR "productivity" OR "remote" OR "work" OR "future of work" after:2021-01-12 and before:2021-01-14&sort=date:r:2021-01-12:2021-01-14&num=200
horizon = after:2021-01-12 and before:2021-01-14




,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-01-12,Impact of Coronavirus Disease (COVID-19) Pande...,"Jan 12, 2021 · We studied the impact of COVID-...",https://www.frontiersin.org/articles/564364,https,www.frontiersin.org,/articles/564364,,,frontiersin.org,134.213.70.247,GB,United Kingdom,England,Welling,NaN,51.466599,15395.0,None,geolocation,{},134.213.70.247,Impact of Coronavirus Disease (COVID-19) Pande...,"Jan 12, 2021 · We studied the impact of COVID-..."
1,2021-01-12,It's time to reimagine where and how work will...,PwC's second survey into attitudes about remot...,https://www.pwc.com/us/en/library/covid-19/us-...,https,www.pwc.com,/us/en/library/covid-19/us-remote-work-survey....,,,pwc.com,23.75.195.189,US,United States,Georgia,Atlanta,-84.388283,33.798458,16625.0,None,geolocation,{},23.75.195.189,It's time to reimagine where and how work will...,PwC's second survey into attitudes about remot...
